# client instance 생성

In [1]:
DEPLOYMENT_URL="http://10.8.1.229:8000/"
DEPLOYMENT_URL="http://localhost:2024/"

from langgraph_sdk import get_client

client = get_client(url=DEPLOYMENT_URL)

# thread 생성 (앨런의 channel에 대응)

- 생성 요청 후 thread_id를 포함한 데이터를 return 받음
- 하나의 thread에 대해서 여러 에이전트를 호출하고 대화가능
- 현재 앨런은 한개의 channel에 대해서 한개의 agent가 1:1로 대응되므로 현재는 고려할 필요 없지만, 추후에 1개의 채널에 대해서 여러 agent를 호출가능

In [2]:
thread = await client.threads.create()
print(thread)

thread_id = thread["thread_id"]
print(thread_id)

{'thread_id': 'ed5b54e5-a736-4784-a239-57ce53b65dd3', 'created_at': '2025-08-08T01:37:44.870268+00:00', 'updated_at': '2025-08-08T01:37:44.870271+00:00', 'metadata': {}, 'status': 'idle', 'config': {}, 'values': None}
ed5b54e5-a736-4784-a239-57ce53b65dd3


# 에이전트 호출

- 해당 thread에서 어떤 agent를 호출할 지 assistant_id를 포함하여 요청을 보냄

## parameters
- thread_id : 위에서 생성한 thread_id
- assistant_id : 앨런의 persona_id에 대응, agent를 구분하는 id, 슬라이드 에이전트의 경우 "slide_generate_agent"
- input : 해당 agent 요청에 대한 parameter, 현재는 messages만 존재, 앨런 페이지에서 유저로 부터 받는 input을 해당 parameter에 dict로 넣어줌
- stream_mode : 어떤 event에 따라서 메시지를 받을지 지정, updates(한 단계가 끝날 때마나 메시지가 옴), events(llm이 생성하는 단위로 메시지 생성)

In [3]:
def filter_ai_msg(chunk):
    # 유저와 채팅 창에 출력될 ai 메시지를 필터링하는 함수
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'agent' in node_name:
            return True

def filter_silde_data(chunk):
    # slide 관련 데이터를 담은 메시지를 필터링하는 함수
    # 슬라이드 한페이지 단위로 메시지가 옴
    if 'updates' in chunk.event:
        node_name = chunk.data.keys()
        if 'executor' in node_name:
            return True
            print(chunk.data['agent']['messages'][-1]['content'])

In [4]:
assistant_id = "slide_generate_agent"

input = { "messages": "안녕"}
list_chunk = list()

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
안녕하세요! 슬라이드를 생성하는 데 필요한 도움을 드리겠습니다. 슬라이드 주제는 어떤 것인가요?


In [5]:
assistant_id = "slide_generate_agent"
list_chunk = list()

input = { "messages": "제주도 여행을 주제로 슬라이드를 만들어줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########
제주도 여행을 주제로 슬라이드를 만들기 위해 몇 가지 정보를 추가로 여쭤보고 싶습니다.

1. 총 슬라이드 장 수는 몇 장으로 할까요?
2. 특별히 강조하고 싶은 제주도의 명소나 활동이 있나요?
3. 슬라이드의 대상 관객은 누구인가요? (예: 학생, 가족, 관광객 등)
4. 추가하고 싶은 이미지나 그래픽 요소가 있나요?
5. 특정한 디자인 스타일이나 테마가 있으면 말씀해 주세요.

이 정보들을 알려주시면 더욱 맞춤화된 슬라이드를 만들어 드릴 수 있습니다!


In [ ]:
assistant_id = "slide_generate_agent"
list_chunk = list()

input = {"messages": "알아서 해줘"}

async for chunk in client.runs.stream(
    thread_id,
    assistant_id=assistant_id,
    input=input,
    stream_mode=["updates"],
    stream_subgraphs=True
):
    try:
        list_chunk.append(chunk)
        if filter_ai_msg(chunk):
            print("#" * 10 + "  AI 메시지  " + "#" * 10)
            print(chunk.data['agent']['messages'][-1]['content'])
        elif filter_silde_data(chunk):
            print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
            print(chunk)
    except:
        continue
    pass

##########  AI 메시지  ##########

##########  AI 메시지  ##########

##########  AI 메시지  ##########
Here's a detailed report plan for a guide on a trip to Jeju Island:

### 2. 개요
#### Name: 제주도 소개 및 여행 개요
- **Description**: 제주도의 역사, 문화, 기후, 그리고 여행의 전반적인 개요에 대한 소개.
- **Research**: 제주도의 역사와 문화를 간략히 설명하는 자료를 검토합니다.
- **Content**: ""
- **Img**: ""
- **HTML**: ""

### 3. 명소
#### Name: 제주도의 명소
- **Description**: 한라산, 성산일출봉 등 제주도의 주요 관광 명소와 자연경관에 대해 설명합니다.
- **Research**: 제주도의 인기 있는 관광지와 각 지역별 특색있는 명소 정보를 조사합니다.
- **Content**: ""
- **Img**: ""
- **HTML**: ""

### 4. 전통 음식
#### Name: 제주 전통 음식
- **Description**: 흑돼지, 전복구이 등 제주에서 꼭 맛봐야할 전통 음식을 소개합니다.
- **Research**: 제주도의 전통 음식과 관련된 자료를 조사하여 지역 음식 문화를 소화합니다.
- **Content**: ""
- **Img**: ""
- **HTML**: ""

### 5. 숙박
#### Name: 제주도의 숙박 시설
- **Description**: 호텔, 게스트하우스, 리조트 등의 다양한 숙박 옵션과 그 특징을 확인합니다.
- **Research**: 제주도 숙박 시설의 다양한 옵션과 리뷰, 가격대 등을 분석합니다.
- **Content**: ""
- **Img**: ""
- **HTML**: ""

### 6. 교통 정보
#### Name: 제주도의 교통
- **Description**: 버스 및

In [ ]:
for chunk in list_chunk:
    if filter_silde_data(chunk):
        print("#" * 10 + "  슬라이드 데이터  " + "#" * 10)
        print(chunk.data)
        break

In [ ]:
state = await client.threads.get_state(thread_id=thread_id)

for msg in state["values"]["messages"]:
    if msg["type"] == "human":
        print("Human :", msg["content"])
    else:
        print("AI :", msg["content"])

In [ ]:
state["values"]["slides"]